<a href="https://colab.research.google.com/github/yajima-yasutoshi/DataScience2025/blob/main/20251216/%E3%82%AF%E3%83%A9%E3%82%B9%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラスタリング

20251216

# この講義の目的

Pythonを用いたクラスタリング手法の理解

# 環境の準備

まず初めに、日本語を表示するための準備と
分析に必要なライブラリーのインポートを行うため、
以下のセルを実行する。

In [ ]:
# インストール
!pip install japanize-matplotlib

# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# Pandasの表示形式を小数点以下1桁にする
pd.options.display.float_format = '{:.1f}'.format

# データの準備
今回利用するデータを準備する。
本日の講義資料が置かれたサイトから
「利用回数.xlsx」というファイルをいったん各自のPCダウンロードし、
そのファイルをGoogle Drive にアップロードをする。

アップロードができたら、
Google Colab の環境に Google Drive がマウントされていることと、
さらに、
このファイルが保存されていることを
Google Colab の環境から確認する。


以下のセルでは、Google Drive のマウント実行する。


In [ ]:
# Google Drive のマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

保存したファイルをデータフレーム変数 df に読み込む。 なお、ファイルパスは、個人の環境に合わせて変更が必要である。

In [ ]:
# 読み込むファイルを指定する
file_path = '/content/drive/MyDrive/データマイニング/利用回数.xlsx'
df = pd.read_excel(file_path)

## データの確認

info()
を用いて、データの列に関する情報を出力します。

In [ ]:
df.info()

「利用回数.xlsx」には、顧客ID、年齢、利用回数
という数値型の項目が 3 つあることが確認できます。

また、head() を使ってデータの先頭を表示します。

In [ ]:
df.head()

In [ ]:
df.describe()

# クラスタリングとは

## 散布図による可視化

年齢と利用回数の間の関係を可視化する。
どちらも数値型の項目なので、
**散布図**を使う。

In [ ]:
sns.scatterplot(x='年齢', y='利用回数', data=df)
plt.grid(True)  # グリッド線を追加
plt.title('散布図')
plt.show()

上の散布図から、全体の利用者は
 * 年齢が高く利用回数の多い利用者
 * 年齢が低く利用回数の多い利用者
 * 年齢が低く利用回数の少ない利用者

に傾向が分かれていることが観察できる。

傾向が似通っているレコード(この例では利用者)を似た性質のグループに分けて、
全体をいくつかのまとまりとして理解する手法が
**「クラスタリング」**である。

スーパーやドラッグストア、コンビニなどの小売店などでは、
顧客の属性（性別や年齢など）や
購買行動（来店回数や利用回数、購買金額など）のデータを元にして、
傾向が似ている顧客をいくつかのグループ（クラスター）に分類し、
どのような顧客が良く利用しているのか、
どのような商品を購入する顧客がいるのか、
など、顧客の行動を理解することで販促や商品開発などを行っている。

参考事例：
https://www.nikkei.com/article/DGXZQOUC132X30T10C22A9000000/

# データを標準化する

 クラスタリングを実行する事前の準備として、
 データの**標準化**が必要である。
 標準化とは、データの大きさを平均 0、分散 1 に変換することである。
なお、分散の平方根が標準偏差となることから、
分散が1となれば標準偏差も1になる。

アップロードしたデータの分布を確認する。
年齢と利用回数の分布は以下のようである。

In [ ]:
fig, (ax1, ax2) = plt.subplots( 1, 2 , figsize=(10,4))
sns.histplot( df['年齢'], bins=10, kde=False, ax=ax1)
sns.histplot( df['利用回数'], bins=10, kde=False, ax=ax2)
plt.show()

上のグラフからも明らかなように、
年齢と利用回数では値のばらつきの範囲が大きく異なっている。
実際に分散を計算すると
両者のばらつきが
大きく異なっていることが確認できる。

In [ ]:
df['年齢'].var()

In [ ]:
df['利用回数'].var()

このように、項目によって数値の大きさや散らばりが異なる場合には、
**標準化**が不可欠である。

## Pythonでの標準化の方法

データ分析用のライブラリである sklearn には
データを変換するたモジュールが用意されている。
データの標準化には、
StandardScaler() を用いる。

インポートは以下のように行う。
```
sklearn.preprocessing import StandardScaler
```


StandardScaler は、以下の3つのステップで実行する。

1. 準備：適当な名前で変数を用意
2. 訓練（fit） ：変換に必要な情報（標準化の場合であれば、元のデータの平均と分散）の計算
3. 変換（transform）：実際に変換を実施する


In [ ]:
# 標準化に必要なライブラリーのインポート
from sklearn.preprocessing import StandardScaler

# 標準化の対象となる項目を変数 X にセットする
X = df[['年齢', '利用回数']]

# １．準備
scaler = StandardScaler()

# ２．訓練の実施
scaler.fit(X)

# ３．標準化の実施
# 標準化後のデータを再びXにセットする
X = scaler.transform(X)

変換に用いたパラメータ（平均や分散）を確認することもできる。


*   平均：scaler.mean_
*   分散：scaler.var_



In [ ]:
scaler.mean_

In [ ]:
scaler.var_

変数 X には2項目のデータ（年齢と利用回数）が格納されているので、
項目ごとに平均と分散が表示される。

## 可視化して確認する

データの標準化が行われた様子を可視化して確認する。
変換後の散布図では、
縦軸と横軸の値のばらつきの範囲が等しいことが分かる。

In [ ]:
# 標準化の確認
X = pd.DataFrame( X, columns=['x1', 'x2'])

fig, (ax1, ax2) = plt.subplots( 1, 2 , figsize=(10,4))

ax1.set_title("変換前")
sns.scatterplot(data=df, x='年齢', y='利用回数', ax=ax1 )
ax1.grid(True)  # グリッド線を追加

ax2.set_title("変換後")
sns.scatterplot(data=X, x='x1', y='x2', ax=ax2)
ax2.grid(True)  # グリッド線を追加

plt.show()

##数値で確認する

変換後の平均と分散(標準偏差)を確認する。


In [ ]:
X.describe()

# クラスタリングの実行

## K-means 法

本講義では、クラスタリングを行う代表的な手法であるK-means法を扱う。

データ分析ライブラリーの sklearn に含まれている **KMeans** を用いる。
使い方は、


1.   準備
2.   計算（fit）

の2ステップである。

準備では、いくつかのパラメータを設定する。主なものは、
*   n_clusters
*   init
*   max_iter
*   n_init

である。

中でも、もっとも重要なパラメータは
クラスタ数を指定する

n_clusters

である。



詳しい使い方は以下のリンクを参照

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


K-means法では、まずクラスタ数を決める必要がある。
以下のコードでは、
仮にクラスタ数を3として実行している。

In [ ]:
# クラスタリングに必要なライブラリーのインポート
from sklearn.cluster import KMeans

# 仮にクラスタ数を3として実行する
n = 3

# １．準備
kmeans = KMeans(n_clusters = n, init='k-means++', max_iter=500, n_init='auto')

# ２．クラスタの計算
kmeans.fit(X)

In [ ]:
print(kmeans)

#クラスタ結果の可視化

クラスタリングの結果は、
kmeans.labels_ に格納されている。
クラスタ数を 3 に設定したので、
kmeans.labels_ には、0、1、2 のいずれかの数字が格納されている。

結果を、元データのデータフレーム変数 df に追加することで、
可視化が容易になる。
以下のセルでは、
クラスタリングの結果をデータフレームに 'cluster' という列で追加している。

In [ ]:
# クラスタラベルをデータフレームに追加
df['cluster'] = kmeans.labels_

In [ ]:
df.head()

年齢と利用回数の散布図を、クラスタで層別（色分け）して可視化する。

In [ ]:
# 結果の可視化（seabornを使用）
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
plt.grid(True)  # グリッド線を追加
plt.show()

## クラスター平均

クラスター毎にデータを平均した点をクラスター平均
（クラスター重心、クラスター中心）と呼ぶ。

基礎集計で説明した groupby を使って計算する。
すなわち、
分類の項目を「cluster」とし、
「年齢」と「利用回数」の項目の平均を計算する。

In [ ]:
# クラスタで分類して年齢と利用回数の平均を求める
# クラスタ中心と呼ぶ
df.groupby('cluster')[['年齢','利用回数']].mean()

上で計算したクラスタ中心は、
それぞれのクラスターを代表するデータと考えることができ、
クラスターの特徴を解釈する際に利用する。

例えば、クラスタ0は、
年齢も高く利用回数も多い顧客の集まりだと解釈をする。

また、各顧客はクラスタ中心の中で最も距離が短いクラスタに属すように
決められている。


クラスタ中心を●で可視化すると以下のようになる。

In [ ]:
c = df.groupby('cluster')[['年齢','利用回数']].mean().reset_index()
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
#sns.scatterplot(x='年齢', y='利用回数', data=c, color='k', s=100)
sns.scatterplot(x='年齢', y='利用回数', data=c, s=100, hue='cluster', palette='Set1')
plt.grid(True)  # グリッド線を追加
plt.show()

# クラスタ数の決定

今までの例では、
先にクラスタ数を決めてからクラスタリングを実行した。
ここでは、クラスタ数はどのように決めればよいか説明する。

## クラスタ内距離
まず、計算されたクラスターに
どの程度データがまとまっているかを表す指標に
**クラスタ内距離**
がある。

これは、
クラスタ毎にクラスタ中心から各データへの距離の二乗の和を計算したもので、
クラスタ内距離が小さいほど、クラスタ中心にデータが集中していることを表す。

上のプログラムを実行した場合には、
kmeans.inertia_
という変数に格納されている。

In [ ]:
# クラスタ毎にクラスタ中心から各データへの距離の二乗の和
kmeans.inertia_

例えば、
クラスタ数を 4 に変更してクラスタリングを実行し、
クラスタ内距離を確認する。

In [ ]:
# クラスタ数を 4 にして実行する
best_n_clusters = 4

kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=500, n_init='auto')
kmeans.fit(X)
df['cluster'] = kmeans.labels_

# 図示する
c = df.groupby('cluster')[['年齢','利用回数']].mean().reset_index()
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
sns.scatterplot(x='年齢', y='利用回数', data=c, color='k', s=100)
plt.grid(True)  # グリッド線を追加
plt.show()

In [ ]:
# クラスタ毎にクラスタ中心から各データへの距離の二乗の和
kmeans.inertia_

クラスタ数が 3 の場合では、クラスタ内距離が約35であった。
それと比べるとクラスタ内距離が減少することが確認できる。

一般に、クラスタ数を増やすとクラスタ内距離を減少させることができるものの、
一方で、同程度のクラスタ内距離であればクラスタ数が少なくシンプルなクラスタリングが良いとされる。

## エルボー法

クラスタ内距離とクラスタ数のバランスの良いものを
決める方法として、エルボー法が使われる。

クラスタ数を変化させながら、クラスタ中心からの距離の二乗の和を計算しグラフ化する。
グラフの減少度合いが変化し始めるところを、最適なクラスター数として採用する。

こういった作業はAIの開発では一般的で、**「ハイパーパラメータチューニング」**と呼ばれる。

In [ ]:
# Elbow Methodによるハイパーパラメータチューニング
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init='auto')
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
# グリッド線を表示するスタイルに変更する
sns.set_style('whitegrid')
# sns.set_theme('paper')
sns.set(font='IPAexGothic')

# エルボー図のプロット
elbow = pd.DataFrame( inertia, columns = ['Inertia'])
elbow['Num of clusters'] = range(1,11)
sns.lineplot(data = elbow, x='Num of clusters', y= 'Inertia')

上のグラフから、クラスタ数が3のところで
グラフの傾きが大きく変化しており、
最適なクラスタ数は3であると判断する。

In [ ]:
# データをを変数 X にセットする
X = df[ ['年齢', '利用回数']]

# 準備の準備
scaler = StandardScaler()

# 標準化後のデータを再びXにセットする
X = scaler.fit_transform(X)

# クラスタ数を3として実行する
best_n_clusters = 3

# 準備
kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=500, n_init='auto')

# クラスタの計算
kmeans.fit(X)

# クラスタラベルをデータフレームに追加
df['cluster'] = kmeans.labels_

# クラスタで分類して年齢と利用回数の平均を求める
sns.set_style('whitegrid')
sns.set(font='IPAexGothic')
c = df.groupby('cluster')[['年齢','利用回数']].mean().reset_index()
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
sns.scatterplot(x='年齢', y='利用回数', data=c, color='k', s=100)

---
---


#小テスト

それでは、以上の内容の確認するための小テストに取組む。

**講義の時間内に必ず提出すること**




---



---



#次回の講義に向けて

上で説明したクラスタリングを別のデータで実行する。

用いるデータは「Wineデータ」と呼ばれるものである。

簡単にWineデータに関して説明する。

## データの読み込み

まず、Wineデータをデータフレームにセットするために
以下のセルを実行する。
wine_df という変数にセットされる。

In [ ]:
from sklearn.datasets import load_wine
# データの読み込み
wine = load_wine()
wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)

## データの概要

データの列を確認する。

In [ ]:
wine_df.info()

Wineデータセットは、イタリアの同じ地域で栽培されたワインに
含まれる成分を測定したデータで、全部で13項目ある。

| 項目                          | 説明                                       |
|---------------------------------|--------------------------------------------|
| Alcohol（アルコール）           | ワインに含まれるアルコールの量。           |
| Malic Acid（リンゴ酸）          | ワインに含まれるリンゴ酸の量。             |
| Ash（灰分）                     | ワインの灰分の量。                         |
| Alcalinity of Ash（灰分のアルカリ度） | 灰分のアルカリ度。                   |
| Magnesium（マグネシウム）       | ワインに含まれるマグネシウムの量。         |
| Total Phenols（総フェノール）   | ワインに含まれるフェノール類の総量。       |
| Flavanoids（フラバノイド）      | ワインに含まれるフラバノイドの量。         |
| Nonflavanoid Phenols（非フラバノイドフェノール） | ワインに含まれる非フラバノイドフェノールの量。 |
| Proanthocyanins（プロアントシアニン） | ワインに含まれるプロアントシアニンの量。 |
| Color Intensity（色の強度）     | ワインの色の強度。                         |
| Hue（色相）                     | ワインの色相。                             |
| OD280/OD315 of Diluted Wines（希釈ワインのOD280/OD315） | 希釈されたワインのOD280/OD315の比率。 |
| Proline（プロリン）             | ワインに含まれるプロリンの量。             |

分析を簡単にするため以下の処理を実行し
項目を限定する。


In [ ]:
wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)
# 相関行列を計算
correlation_matrix = wine_df.corr().abs()
# sns.heatmap(correlation_matrix, cmap= sns.color_palette('coolwarm', 10), annot=True,fmt='.2f', vmin = -1, vmax = 1)

# 相関が0.6以上の変数を取得
high_corr_var = {}
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= 0.5:
            colname = correlation_matrix.columns[i]
            high_corr_var[colname] = abs(correlation_matrix.iloc[i, j])

# 相関性の高い変数を削除
wine_df.drop(columns=high_corr_var.keys(), inplace=True)

# 結果の表示
# print("Remaining columns after removing highly correlated variables:")
print(wine_df.columns)
wine_df.info()

## 次回の講義までの課題

データフレーム wine_df に対して
クラスタリングを実行する。

できるだけ講義時間内で行う。
